### [ref: python如何画出漂亮的地图？](https://www.zhihu.com/question/33783546)

In [1]:
import os
import folium
import pandas as pd
from folium import plugins
from folium.plugins import HeatMap
from conf import config
from collections import defaultdict

In [2]:
CITY_NAME = '上海'  # change this variable to select citys

In [3]:
data_dir = os.path.join(config.data_dir, CITY_NAME)
city_longitude = config.AMAP_CITY_DICT[CITY_NAME]['lng']
city_latitude = config.AMAP_CITY_DICT[CITY_NAME]['lat']

### load data

In [4]:
df_basic = pd.read_excel(os.path.join(data_dir, config.FILE_NAME_DICT['basic']))
df_basic.head()

,line_name,line_name_add,line_color,station_name,station_pinyin,station_idx,station_lng,station_lat
0,1号线,default,#E3002A,莘庄,XinZhuang,1,121.385373,31.111152
1,1号线,default,#E3002A,外环路,WaiHuan Lu,2,121.393020,31.120899
2,1号线,default,#E3002A,莲花路,LianHua Lu,3,121.402910,31.130900
3,1号线,default,#E3002A,锦江乐园,JinJiangLeYuan,4,121.414107,31.142217
4,1号线,default,#E3002A,上海南站,ShangHai NanZhan,5,121.430041,31.154579


In [5]:
df_adj = pd.read_excel(os.path.join(data_dir, config.FILE_NAME_DICT['adj']))
df_adj.head()

,station_name,num_adjst_1km,adjst_1km,num_adjst_3km,adjst_3km
0,一大会址·新天地,3,"{'一大会址·黄陂南路': 0.86, '老西门': 0.87, '马当路': 0.79}",24,"{'一大会址·黄陂南路': 0.86, '世博会博物馆': 2.18, '人民广场': 1...."
1,一大会址·黄陂南路,3,"{'一大会址·新天地': 0.86, '大世界': 0.71, '淮海中路': 0.89}",23,"{'一大会址·新天地': 0.86, '人民广场': 1.01, '南京东路': 1.93,..."
2,七宝,0,{},5,"{'七莘路': 2.92, '中春路': 1.28, '星中路': 1.9, '紫藤路': ..."
3,七莘路,0,{},3,"{'七宝': 2.92, '虹莘路': 1.82, '顾戴路': 2.98}"
4,三林,2,"{'上南路': 0.83, '华夏西路': 0.8}",9,"{'三林东': 1.15, '上南路': 0.83, '凌兆新村': 2.1, '华夏西路'..."


In [6]:
df = pd.merge(df_basic, df_adj, on='station_name', how='left')
df.shape

(584, 12)

In [7]:
df['num_adjst_3km_normal'] = df['num_adjst_3km'].apply(
    lambda x: (x -  df['num_adjst_3km'].min()) / ( df['num_adjst_3km'].max() -  df['num_adjst_3km'].min())
)

In [8]:
df_dedup = df.drop_duplicates(subset='station_name', keep='first')
df_dedup.shape

(408, 13)

### plot map

#### scatter, cluster, line

In [9]:
 # tiles: 'OpenStreetMap', 'Stamen Terrain', 'Stamen Toner' and etc.
city_map = folium.Map(location=[city_latitude, city_longitude], zoom_start=10, tiles='OpenStreetMap') 

##### scatter

In [10]:
incidents = folium.map.FeatureGroup()

for lat, lng, relative_rate in zip(df_dedup.station_lat, df_dedup.station_lng, df_dedup.num_adjst_3km_normal):
    incidents.add_child(
        folium.Circle([lat, lng], radius= 10 + 300 * relative_rate, color='magenta', fill=True, fill_opacity=1.0)
    )

city_map.add_child(incidents)

##### cluster

In [11]:
incidents = plugins.MarkerCluster()

for lat, lng, station_name in zip(df_dedup.station_lat, df_dedup.station_lng, df_dedup.station_pinyin):
    incidents.add_child(
        folium.Marker([lat, lng], icon=None, popup=station_name)
    )

city_map.add_child(incidents) 

##### line

In [12]:
line_groups = list(map(list, df[['line_name', 'line_name_add']].values))
line_groups_dedup = []
for line_group in line_groups:
    if line_group not in line_groups_dedup:
        line_groups_dedup.append(line_group)
len(line_groups_dedup)

23

In [13]:
points_dict = defaultdict(lambda: dict())
for line_group in line_groups_dedup:
    key = '-'.join(line_group)
    tmp_df = df[(df['line_name'] == line_group[0]) & (df['line_name_add'] == line_group[1])]
    points_dict[key]['points'] = tmp_df[['station_lat', 'station_lng']].values
    points_dict[key]['color'] = tmp_df['line_color'].values[0]

In [14]:
for key, val in points_dict.items():
    folium.PolyLine(val['points'], color=val['color'], weight=2.5, opacity=1).add_to(city_map)

In [15]:
city_map.save(os.path.join(data_dir, 'citymap_cluster.html'))
city_map

#### heatmap

In [16]:
 # tiles: 'OpenStreetMap', 'Stamen Terrain', 'Stamen Toner' and etc.
city_map = folium.Map(location=[city_latitude, city_longitude], zoom_start=10, tiles='OpenStreetMap') 

In [17]:
# Convert data format
heatdata = df_dedup[['station_lat','station_lng']].values.tolist()
# add incidents to map
HeatMap(heatdata).add_to(city_map)

city_map.save(os.path.join(data_dir, 'citymap_heatmap.html'))
city_map